##  PREDICTIVE MODELING
### LOGISTIC REGRESSION FOR PREDICTIVE MODELING
### Jaouad Safouani 

In [ ]:
#importing libraries
import pandas as pd 
# display all columns in the dataframe.
import numpy as np
#format numbers
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
plt.rc("font", size=14)
%matplotlib inline
import seaborn as sns 
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
import statsmodels.api as sm
# from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# importing math library
import math
from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix, accuracy_score 
from statsmodels.stats.outliers_influence import variance_inflation_factor 

In [ ]:
df= pd.read_csv('churn_clean.csv')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
print(sorted(list(df.columns)))

In [ ]:
df.info()

In [ ]:
var_to_be_dropped_list = ['CaseOrder', 'Customer_id', 'Interaction', 'UID','TimeZone','Job','County'
                          ,'State','City','Zip', 'Lat','Lng','Population', 'Area']

In [ ]:
#dropping columns
df.drop(columns = var_to_be_dropped_list , inplace = True)


In [ ]:
# Rename last 8 survey columns to the appropriate description per the data dictionary
df.rename(columns = {'Item1':'TimelyResponse', 
                     'Item2':'Fixes', 
                     'Item3':'Replacements', 
                     'Item4':'Reliability', 
                     'Item5':'Options', 
                     'Item6':'Respectfulness', 
                     'Item7':'Courteous', 
                     'Item8':'Listening'}, 
          inplace=True)

#### Count of Churn

In [ ]:
ax = sns.countplot(x= "Churn", data= df,
    order = df["Churn"].value_counts().index)
for p, label in zip(ax.patches, df["Churn"].value_counts()):
    ax.annotate(label, (p.get_x()+0.375, p.get_height()+0.15))

In [ ]:
#mean of all variables by churn
df.groupby('Churn').mean()

In [ ]:
# Independent_variables = [col for col in df.columns if col!='Churn']

In [ ]:
# y = df[Independent_variables]
# X = df['Churn']

In [ ]:
# print(Independent_variables)

In [ ]:
df.shape

### Distribution of categorical and numerical vars

In [ ]:
# create a dictionary with columns and their datatype. 
dict_col_type = dict()
for col in list(df.columns):
    dict_col_type[col] = df[col].dtypes

In [ ]:
dict_col_type

In [ ]:
df_numeric_variable= dict()
df_object_variable =dict()
for i, v in dict_col_type.items():
    if (dict_col_type[i]=='int64' or dict_col_type[i]=='float64'):
        df_numeric_variable[i]= v
    else:
        df_object_variable[i] = v

In [ ]:
# df_object_variable
df_numeric_variable

In [ ]:
var_obj  = sorted(list(df_object_variable.keys()))
var_obj_cnt = int(len(var_obj)/3)

In [ ]:
var_obj_cnt

In [ ]:
#distribution of all categorical variables in df

fig, axes = plt.subplots(nrows = var_obj_cnt ,ncols = 3,
figsize = (20,25))
for i, item in enumerate(var_obj):
    if i < 6:
        ax = df[item].value_counts().plot(
        kind = 'bar',ax=axes[i,0],
        rot = 15 )
        
    elif i >=6 and i < 12:
        ax = df[item].value_counts().plot(
        kind = 'bar',ax=axes[i-6,1],
        rot = 15, color ='orange')
        
    elif i >= 12 and i<18:
        ax = df[item].value_counts().plot(
        kind = 'bar',ax=axes[i-12,2],
        rot = 15)
    for p, label in zip(ax.patches, df[item].value_counts()):
        ax.annotate(label, (p.get_x() +0.15, p.get_height()+.4))
        
    ax.set_title(item, fontsize =14)
# plt.annotate(df[item].value_counts())
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.89, 
                    top=0.89, 
                    wspace=0.2, 
                    hspace=0.65)


plt.savefig("CountOfCategoricalVariables.png")    

#### Describe df

In [ ]:
df.describe()

By calling df.describe(), we notice that on average a customer is 53 years old has 2 children with 10 seconds of outage per week and income of 39806.926771 and emailed customer service 12 times.
We also notice that 75% of the customers has 3 children and are 71 years old and have about 12 second of outage per week with and income of 39806.926771 and emailed customer service 14.
With standard deviation for children 2.1472 and Age 20.69 and 2.97 second of outage per week and income of 28199.916702 and email 3.02.


### Numerical Variables distribution

In [ ]:
var_num_list = sorted(list(df_numeric_variable.keys()))
len(var_num_list)

In [ ]:
var_num_list = sorted(list(df_numeric_variable.keys()))
sns.set(font_scale=1.23)
fig, axes = plt.subplots(nrows = int(len(var_num_list)/3) ,ncols = 3,
figsize = (20,20))
for i, item in enumerate(var_num_list):
    if i < 6:
        sns.boxplot(x= item, data = df,showmeans=True,ax=axes[i,0])
        
    elif i >=6 and i < 12:
        sns.boxplot(x= item, data = df,showmeans=True,ax=axes[i-6,1],color ='orange')
        
    elif i >= 12 and i<18:
         sns.boxplot(x= item, data = df,showmeans=True,ax=axes[i-12,2],color ='#feefc0')

        
#     ax.set_title(item, fontsize =14)
    
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=1, 
                    wspace=0.2, 
                    hspace=0.3)

plt.savefig("IdentifyingOutliersNumericalVar.png") 

After Investigation the outlies they deemed to be acceptable

## Bivariate Analysis

### Categorical by Churn

In [ ]:
bivariate_exclude_chrun = [col for col in var_obj if col !='Churn']


In [ ]:


fig, axes = plt.subplots(nrows = var_obj_cnt ,ncols = 3,
figsize = (22,25))

for i, item in enumerate(bivariate_exclude_chrun):
    cat_churn = df.groupby([item,'Churn']).size().unstack()
    # Transpose the columns into rows than devide each row value by the total of the each column 
    # to calculte of percetage of column
    if i < 6:
        ax = (cat_churn.T*100.0 / cat_churn.T.sum()).T.plot(kind='bar',
                                                                      ax=axes[i,0],
                                                                      stacked = True,
                                                                      rot =15,
                                                            figsize = (25,40))

        ax.yaxis.set_major_formatter(mtick.PercentFormatter())

        for p in ax.patches:
            width, height = p.get_width(), p.get_height()
            x, y = p.get_xy() 
            ax.text(x+width/2, 
                    y+height/4, 
                    '{:.1f}%'.format(height), 
                    horizontalalignment='center', 
                    verticalalignment='center')
        ax.autoscale(enable=False, axis='both', tight=False)
        
    elif i >=6 and i < 12:
        ax = (cat_churn.T*100.0 / cat_churn.T.sum()).T.plot(kind='bar',
                                                                      ax=axes[i-6,1],
                                                                      stacked = True,
                                                                      rot =15,
                                                            figsize = (25,40))

        ax.yaxis.set_major_formatter(mtick.PercentFormatter())

        for p in ax.patches:
            width, height = p.get_width(), p.get_height()
            x, y = p.get_xy() 
            ax.text(x+width/2, 
                    y+height/4, 
                    '{:.1f}%'.format(height), 
                    horizontalalignment='center', 
                    verticalalignment='center')
        ax.autoscale(enable=False, axis='both', tight=False)
        

    elif i >= 12 and i<17:
        ax = (cat_churn.T*100.0 / cat_churn.T.sum()).T.plot(kind='bar',
                                                                      ax=axes[i-12,2],
                                                                      stacked = True,
                                                                      rot =15,
                                                            figsize = (25,40))

        ax.yaxis.set_major_formatter(mtick.PercentFormatter())

        for p in ax.patches:
            width, height = p.get_width(), p.get_height()
            x, y = p.get_xy() 
            ax.text(x+width/2, 
                    y+height/4, 
                    '{:.1f}%'.format(height), 
                    horizontalalignment='center', 
                    verticalalignment='center')
        ax.autoscale(enable=False, axis='both', tight=False)

plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.89, 
                    top=1, 
                    wspace=0.2, 
                    hspace=0.4)


plt.savefig("BivariateAnalysis_CategoricalByChurn.png")    

 * We conclude from the bivariate analysis that the month to month contract has more churn at 37.3% compared to the other contract types
 * DSL has more churn at 32.2% compared to other internet services.
 * people that stream movies tend to churn compared to people who do not.
 * the rest of categorical variables tend to have similar stats by churn

### Distribution of Numerical Variables by Churn

In [ ]:
df_var_numcnt = math.ceil(len(var_num_list)/3)
sns.set(font_scale=1.23)

fig, axes = plt.subplots(nrows = df_var_numcnt ,ncols = 3,
figsize = (22,25))
for i, item in enumerate(var_num_list):
    if i < 6:
        sns.histplot(x= item, data = df,ax=axes[i,0], hue ='Churn',multiple="stack")
        
    elif i >=6 and i < 12:
        sns.histplot(x= item, data = df,ax=axes[i-6,1], hue ='Churn',multiple="stack")
        
    elif i >= 12 and i<18:
         sns.histplot(x= item, data =df,ax=axes[i-12,2], hue ='Churn',multiple="stack")
            
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.2, 
                    hspace=0.3)

plt.savefig("BivariateHistogram_NumericalByChurn.png") 

We conlude from the the distirubution of the numerical variables bu churn the following:
* Clients that churned  are client that send most emails and have larger outage sec perweek even they had Timely Response and Options and customer was curteous.
* Churned customer has less Tenure and had fewer average data usage per year.
* Churned Customers are from all ages, there is a uniform distributiion regradless of if a customer churned or not.

### variance_inflation_factor on Numerical variables

Let's check for multicollinearity

In [ ]:
# variance_inflation_factor
df_new_model = df[var_num_list]
vif = pd.DataFrame()
vif['Features'] = var_num_list
vif['VIF'] = [variance_inflation_factor(df_new_model.values, i) for i in range(df_new_model.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif


by Using VIF we notice that
* Bandwidth_GB_Year	312.45
* Tenure	246.96
seemed to be correlated let's verif them in the correlation matrix and drop one of them
that goes for 
* TimelyResponse	27.23
* Fixes	24.03
they seem to be correlated 
that goes for 
* MonthlyCharge	21.88
* Replacements	19.82
they seem to be correlated 

### Correlation Matrix

In [ ]:

f, ax = plt.subplots(figsize=(25, 25))
df_chrun_corr= df[var_num_list]
sns.heatmap(df_chrun_corr.corr(), annot=True,cmap="YlGnBu",square=True, vmax=1)
plt.title("Independent variables Correlation Matrix", fontsize = 24)
plt.savefig("IndependentVarialbesCorrMatrix.png")  

The correlation matrix confirm that:
    
Bandwidth_GB_Year is highly correlated with Tenure: 0.99 we will need to drop the Tenure as it makes more sense to use Bandwidth_GB_Year as in input to predict customer that are going to churn per research question

TimelyResponse is correlated with both Fixes: 0.66 and with Replacements at 0.58, also there is a correlation between Replacement and Fixes. We will drop both Fixes and Replacements.
MonthlyCharge and Replacements are not correlated. 
Conclusion:


In [ ]:
multi_corre_var_to_be_droped = ['Tenure', 'Replacements','Fixes']

In [ ]:
df.drop(columns = multi_corre_var_to_be_droped, inplace =True)

In [ ]:
#verify that the columns were dropped
print(sorted(list(df.columns)))

## Re-expression

In [ ]:

# Get the columns with string values and a lis the numerical columns.
list_object_col  =[]
list_numeric_col =[]
for col , _type in dict_col_type.items():
    if 'object' in str(_type):
        list_object_col.append(col)
    else:
        list_numeric_col.append(col)
#Assign unique values for each object variable and store data in a dictionary        
dict_col_withdistinct_values=dict()
for col in list_object_col:
    dict_col_withdistinct_values[col]= df[col].unique()

In [ ]:
for key,values in dict_col_withdistinct_values.items():
    print(key," : ", values)

In [ ]:
# identify columns with Yes/ NO 
# We will use ordinal Encoding for variables with Yes No 
# and One hot Encoding for the other categorical non ordinal columns.
List_var_Yes_No=[]
List_var_Other=[]
for key,values in dict_col_withdistinct_values.items():
    if 'No' in values and 'Yes' in values:
        List_var_Yes_No.append(key)
    else:
        List_var_Other.append(key)
        

In [ ]:
List_var_Other

### Ordinal Encoding

In [ ]:
print(sorted(List_var_Yes_No))

In [ ]:
# Defining the map function to replace Yes and No with 1 and 0
# Ordinal Encoding
def Yes_No_dict_map(x):
    ''' Ordinal Encoding '''
    return x.map({'Yes':1, 'No':0})


In [ ]:
# Replace the Yes and No with 1 and 0 in binary variables.
# Applying Ordinal encoding to Re express Yes No to 1 and 0 respectevely.
df[List_var_Yes_No]=df[List_var_Yes_No].apply(Yes_No_dict_map)

In [ ]:
#Verifying that the encoding was successful
df[List_var_Yes_No].head()

### One-Hot Encoding


In [ ]:
print(sorted(List_var_Other))

In [ ]:
dict_var_Other = dict()
for key,values in dict_col_withdistinct_values.items():
    if key in List_var_Other:
        dict_var_Other[key] = values

In [ ]:
df['Gender'].unique()

In [ ]:
# Using One hot encoding to prepare the data multi leaner regresssion model.
# Notice we are dropping the first column in the dummy data frame to avoid the dummy-variable trap, 
# resulting from the One hot encoding.

for col in dict_var_Other.keys():
    df_ = pd.get_dummies(df[col], drop_first = True).astype('int64')
    df = pd.concat([df, df_], axis = 1)
    df.drop(columns = col, inplace = True)
    

In [ ]:
#dropped None as it is an outliers.
df.drop(columns=['None'], inplace =True)

In [ ]:
df.shape

## Bivariate Analysis of the distribution of the Re-expressed variables by Churn

In [ ]:
df_columns = sorted(list(df.columns))

In [ ]:

df_columnscnt =math.ceil(len(df_columns)/3)
df_columnscnt

In [ ]:
# Create distribution plot of all df.
sns.set(font_scale=1.23)
fig, axes = plt.subplots(nrows = df_columnscnt ,ncols = 3,
figsize = (22,35))
for i, item in enumerate(df_columns):
    if i < 14:
        sns.histplot(x= item, data = df,ax=axes[i,0],hue ='Churn')
        
    elif i >=14 and i < 28:
        sns.histplot(x= item, data = df,ax=axes[i-14,1],hue ='Churn')
        
    elif i >= 28and i<40:
         sns.histplot(x= item, data = df,ax=axes[i-28,2],hue ='Churn')
            
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.89, 
                    top=1, 
                    wspace=0.2, 
                    hspace=0.4)

plt.savefig("DistibutionOfRe_expression_By_Churn.png") 

In [ ]:
list_predictors = sorted([col for col in df.columns if col !='Churn'])

In [ ]:
print(list_predictors)

In [ ]:
ordered_df_columns= list_predictors

In [ ]:
ordered_df_columns.append('Churn')
print(ordered_df_columns)

In [ ]:
# make a copy to csv of the oreoared data.
df.to_csv("lr_prepared_churn_clean.csv")

## Logistic Regression Modeling

Lets's set apha = 0.05

Let's split the predicted variable from the predictors.

In [ ]:
y = df['Churn']
y 

In [ ]:
# df.drop(columns=['Churn'], inplace =True)
# X=df
X

### Split the data into training and testing datasets

In [ ]:
#Spliting the data into training and testing subsets randomly.
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y,test_size = 0.3, random_state=30)

In [ ]:
# Print shape of the 4 data sets, training and testing
X_train.shape, X_test.shape, y_train.shape, y_test.shape

### Feature Scaling

Let's scale the features using StandardScaler, the scaler will change the values to vlaues between -1 and 1. This step is necessary for machine learning

In [ ]:
scaler = StandardScaler()

In [ ]:
X_train_copy = pd.DataFrame(scaler.fit_transform(X_train))
X_train_copy.columns = X_train.columns.values
X_train_copy.index = X_train.index.values
X_train = X_train_copy


In [ ]:
X_test_copy = pd.DataFrame(scaler.transform(X_test))
X_test_copy.columns = X_test.columns.values
X_test_copy.index = X_test.index.values
X_test = X_test_copy

In [ ]:
X_test.head()

In [ ]:
X_train.head()

### Logistic Regression - initial model

In [ ]:
initial_model = sm.Logit(endog=y_train, exog=X_train).fit()

In [ ]:

print(initial_model.summary())

In [ ]:
# get odds ratio
np.exp(initial_model.params)

In [ ]:
# get the predicted values for the test dataset [0, 1]
pred = initial_model.predict(exog=X_test)
pred.head()

In [ ]:
# round(pred)

In [ ]:
intial_model_confusion_matrix = confusion_matrix(y_true=y_test, y_pred=list(round(pred)))
intial_model_confusion_matrix

* 1486 True Negative (TN)
* 785 True Positive (TP)
* 10 False Negative (FN)
* 719 False Positive (FP)

In [ ]:
f1score = f1_score(y_true=y_test, y_pred=list(round(pred)))
precisionscore = precision_score(y_true=y_test, y_pred=list(round(pred)))
recallscore = recall_score(y_true=y_test, y_pred=list(round(pred)))
accuracyscore = accuracy_score(y_true=y_test, y_pred=list(round(pred)))

Initial_Model_stats = pd.DataFrame([['Initial Logistic Regression', 
accuracyscore, precisionscore, recallscore, f1score]], columns = ['Model', 
'Accuracy', 'Precision', 'Recall', 'F1 Score'])

Initial_Model_stats

### Equation: Initial Model

In [ ]:
#Generating the MLR Equation for initial Model
LR_equation = []
for var, coef in initial_model.params.items():
    LR_equation.append( '( ' + str(round(coef, 3)) + ' * ' + var  + ' )' )
' + '.join(LR_equation)

### Explanation of the coefficient of the Lr model equation

In [ ]:
variable_exp=[]

for var, coef in initial_model.params.items():
    if var != 'constant':
        if coef >0:
            variable_exp.append( 'For each unit of '  + var +  ' variable, the odds of response of 1 will increase by ' + str(round(coef, 3)) + ' units')
        else:
            variable_exp.append( 'For each unit of '  + var +  ' variable, the odds of response of 0 increase by ' + str(round(abs(coef), 3)) + ' units')
            
variable_exp

## Reduced_model

In [ ]:
# reduced_model.pvalues
# pvalue_reduced = pd.DataFrame(initial_model.pvalues, columns=['pvalue'])
# feature_with_significant_pvalue = sorted(pvalue_reduced[pvalue_reduced['pvalue'] <0.05].index)
# feature_with_significant_pvalue

lets run the variance_inflation_factor to identify correlated independent variables and rid of one of them

In [ ]:
# variance_inflation_factor
reduced_model = X_train
vif = pd.DataFrame()
vif['Features'] = X_train.columns
vif['VIF'] = [variance_inflation_factor(reduced_model.values, i) for i in range(reduced_model.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
# correlation matrix

In [ ]:
f, ax = plt.subplots(figsize=(40, 40))
X_train_chrun_corr= X_train
sns.heatmap(X_train_chrun_corr.corr(), annot=True,cmap="YlGnBu",square=True, vmax=1)
plt.title("Independent variables Correlation Matrix", fontsize = 24)
plt.savefig("ReducedCorrMatrix.png")  

In [ ]:
X_train.drop(columns=['StreamingMovies'], inplace=True)

In [ ]:
X_test.drop(columns=['StreamingMovies'], inplace=True)

### Let's remove the insignificant independent variables

In [ ]:
reduced_model_pvalues = pd.DataFrame(initial_model.pvalues, columns=['pvalues'])
new_predictors = sorted(list(reduced_model_pvalues[reduced_model_pvalues['pvalues']<0.05].index))
new_predictors = [col for col in new_predictors if col !='StreamingMovies' ]

### New training and testing X datasets

In [ ]:
reduced_X_test = X_test[new_predictors]
reduced_X_train = X_train[new_predictors]


### Reduced Model

In [ ]:
reduced_final_model = sm.Logit(endog=y_train, exog= reduced_X_train).fit()
print(reduced_final_model.summary())

#### get the predicted values for the test dataset [0, 1]

In [ ]:
pred_final = reduced_final_model.predict(exog=reduced_X_test)
# pred.head()

####  Reduced_model evaluation

##### reduced_model_confusion_matrix

In [ ]:
reduced_model_confusion_matrix = confusion_matrix(y_true=y_test, y_pred=list(round(pred_final)))
reduced_model_confusion_matrix

##### Model Scores

In [ ]:
f1score = f1_score(y_true=y_test, y_pred=list(round(pred_final)))
precisionscore = precision_score(y_true=y_test, y_pred=list(round(pred_final)))
recallscore = recall_score(y_true=y_test, y_pred=list(round(pred_final)))
accuracyscore = accuracy_score(y_true=y_test, y_pred=list(round(pred_final)))

reduced_Model_stats = pd.DataFrame([['Reduced Logistic Regression', 
accuracyscore, precisionscore, recallscore, f1score]], columns = ['Model', 
'Accuracy', 'Precision', 'Recall', 'F1 Score'])
reduced_Model_stats

In [ ]:
#Generating the MLR Equation for reduced model
LR_equation = []
for var, coef in reduced_final_model.params.items():
    LR_equation.append( '( ' + str(round(coef, 3)) + ' * ' + var  + ' )' )
' + '.join(LR_equation)

In [ ]:
variable_exp=[]

for var, coef in reduced_final_model.params.items():
    if var != 'constant':
        if coef >0:
            variable_exp.append( 'For each unit of '  + var +  ' variable, the odds of response of 1 will increase by ' + str(round(coef, 3)) + ' units')
        else:
            variable_exp.append( 'For each unit of '  + var +  ' variable, the odds of response of 0 increase by ' + str(round(abs(coef), 3)) + ' units')
            
variable_exp

### Model stats

In [ ]:
print(intial_model_confusion_matrix)


Initial Logistic Regression
* 1486 True Negative (TN)
* 785 True Positive (TP)
* 10 False Negative (FN)
* 719 False Positive (FP)

In [ ]:
print(reduced_model_confusion_matrix)

Reduced Logistic Regression
* 1479 True Negative (TN)
* 782 True Positive (TP)
* 13 False Negative (FN)
* 726 False Positive (FP)

In [ ]:
reduced_Model_stats

In [ ]:
model_stats = Initial_Model_stats.append(reduced_Model_stats, ignore_index=True)
model_stats